In [1]:
import pandas as pd
import numpy as np
import scipy as sp
import sklearn as sk

In [2]:
# Store filepath in a variable: use parse_dates=["date"] or dtype=object
# drop the NaN values and note the details of the dropped data.

# 1377884570_tweet_global_warming.csv
# 
gw_1 = "./data/source/GlobalLandTemperatures_GlobalLandTemperaturesByCountry.csv"
# count 544811 avg temp 545550 avg temp uncertain
# Read our Data file with the pandas library
gw_1_df = pd.read_csv(gw_1,parse_dates=["dt"])
gw_1_df.describe()
#w_1_df.info()
#gw_1_df.head()

,AverageTemperature,AverageTemperatureUncertainty
count,544811.000000,545550.000000
mean,17.193354,1.019057
std,10.953966,1.201930
min,-37.658000,0.052000
25%,10.025000,0.323000
50%,20.901000,0.571000
75%,25.814000,1.206000
max,38.842000,15.003000


In [3]:
# drop rows with NaN
gw_1_df= gw_1_df.dropna(axis=0)
gw_1_df.describe()
gw_1_df.info()
gw_1_df.head()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 544811 entries, 0 to 577460
Data columns (total 4 columns):
dt                               544811 non-null datetime64[ns]
AverageTemperature               544811 non-null float64
AverageTemperatureUncertainty    544811 non-null float64
Country                          544811 non-null object
dtypes: datetime64[ns](1), float64(2), object(1)
memory usage: 20.8+ MB


,dt,AverageTemperature,AverageTemperatureUncertainty,Country
0,1743-11-01,4.384,2.294,Åland
5,1744-04-01,1.530,4.680,Åland
6,1744-05-01,6.702,1.789,Åland
7,1744-06-01,11.609,1.577,Åland
8,1744-07-01,15.342,1.410,Åland


In [4]:
# Remove these Åland characters
gw_1_df = gw_1_df.loc[gw_1_df["Country"] != "Åland" ]
gw_1_df.head()

,dt,AverageTemperature,AverageTemperatureUncertainty,Country
3239,1838-04-01,13.008,2.586,Afghanistan
3241,1838-06-01,23.950,2.510,Afghanistan
3242,1838-07-01,26.877,2.883,Afghanistan
3243,1838-08-01,24.938,2.992,Afghanistan
3244,1838-09-01,18.981,2.538,Afghanistan


In [5]:
# Rename columns as short column names
gw_1_df = gw_1_df.rename(columns={'dt':'date',
                  'AverageTemperature':'avg_temp_c',
                  'AverageTemperatureUncertainty':'temp_uncertainty'})

In [6]:
# Reset the index to break the connection to the old data frame
gw_1_df = gw_1_df.sort_values(["date"], ascending=[True]).\
                                        reset_index(drop=True)

In [7]:
# Convert Celsius to Fahrenheit and reorder columns
gw_1_df["avg_temp_f"] = gw_1_df["avg_temp_c"].\
                        apply(lambda x: ((x * 1.8)+32))
gw_1_df = gw_1_df[["date","avg_temp_f","avg_temp_c",
                   "temp_uncertainty","Country"]]

In [8]:
# drop data for 1743 because there is only one month
gw_1_df = gw_1_df.loc[gw_1_df["date"] >= "1744-01-01"]

In [9]:
# Print out the min and max dates
gw1_min_date = gw_1_df["date"].min()
gw1_max_date = gw_1_df["date"].max()
print("min date " + str(gw1_min_date) + " max date " + str(gw1_max_date))

min date 1744-04-01 00:00:00 max date 2013-09-01 00:00:00


In [10]:
gw_1_df.info()
gw_1_df.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 541596 entries, 49 to 541644
Data columns (total 5 columns):
date                541596 non-null datetime64[ns]
avg_temp_f          541596 non-null float64
avg_temp_c          541596 non-null float64
temp_uncertainty    541596 non-null float64
Country             541596 non-null object
dtypes: datetime64[ns](1), float64(3), object(1)
memory usage: 24.8+ MB


,date,avg_temp_f,avg_temp_c,temp_uncertainty,Country
49,1744-04-01,49.1882,9.549,2.289,Guernsey
50,1744-04-01,45.7682,7.649,2.547,Austria
51,1744-04-01,37.7960,3.220,2.551,Saint Pierre And Miquelon
52,1744-04-01,42.2294,5.683,3.679,Latvia
53,1744-04-01,50.1962,10.109,3.105,Ukraine


In [11]:
# Export file as an XLSX or CSV, w/o index, w/ header
gw_1_df.to_csv("./output/clean_GlobalLandTemperaturesByCountry.csv", index=False, header=True)

In [12]:
# create a year column for grouping
gw_1_df["year"] = gw_1_df["date"].dt.year
gw_1_df.head(5)

,date,avg_temp_f,avg_temp_c,temp_uncertainty,Country,year
49,1744-04-01,49.1882,9.549,2.289,Guernsey,1744
50,1744-04-01,45.7682,7.649,2.547,Austria,1744
51,1744-04-01,37.7960,3.220,2.551,Saint Pierre And Miquelon,1744
52,1744-04-01,42.2294,5.683,3.679,Latvia,1744
53,1744-04-01,50.1962,10.109,3.105,Ukraine,1744


In [13]:
# create average yearly temperatures
gw_1avg_df = gw_1_df.groupby(["year","Country"],\
       as_index=False)["avg_temp_f","avg_temp_c","temp_uncertainty"].\
       mean().rename(columns={"avg_temp_f": "avg_yly_tmp_f",
                       "avg_temp_c": "avg_yly_tmp_c",
                       "temp_uncertainty": "avg_yly_tmp_uncertainty"})
gw_1avg_df.head()

,year,Country,avg_yly_tmp_f,avg_yly_tmp_c,avg_yly_tmp_uncertainty
0,1744,Albania,57.311375,14.061875,2.243000
1,1744,Andorra,53.892950,12.162750,2.176625
2,1744,Austria,46.012550,7.784750,2.080250
3,1744,Belarus,46.579325,8.099625,2.417750
4,1744,Belgium,51.225800,10.681000,1.846375


In [14]:
# Export file as an XLSX or CSV, w/o index, w/ header
gw_1avg_df.to_csv("./output/clean_YearLandAvgTempsByCountry.csv", index=False, header=True)

In [16]:
# Store filepath in a variable: use parse_dates=["date"] 
# drop the NaN values and note the details of the dropped data.

# 
gw_2 = "./data/source/GlobalLandTemperatures_GlobalLandTemperaturesByState.csv"
# Read our Data file with the pandas library
gw_2_df = pd.read_csv(gw_2,parse_dates=["dt"])
gw_2_df.info()
gw_2_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 645675 entries, 0 to 645674
Data columns (total 5 columns):
dt                               645675 non-null datetime64[ns]
AverageTemperature               620027 non-null float64
AverageTemperatureUncertainty    620027 non-null float64
State                            645675 non-null object
Country                          645675 non-null object
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 24.6+ MB


,dt,AverageTemperature,AverageTemperatureUncertainty,State,Country
0,1855-05-01,25.544,1.171,Acre,Brazil
1,1855-06-01,24.228,1.103,Acre,Brazil
2,1855-07-01,24.371,1.044,Acre,Brazil
3,1855-08-01,25.427,1.073,Acre,Brazil
4,1855-09-01,25.675,1.014,Acre,Brazil


In [17]:
# Drop rows with NaN
gw_2_df= gw_2_df.dropna(axis=0)
gw_2_df.describe()
gw_2_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 620027 entries, 0 to 645673
Data columns (total 5 columns):
dt                               620027 non-null datetime64[ns]
AverageTemperature               620027 non-null float64
AverageTemperatureUncertainty    620027 non-null float64
State                            620027 non-null object
Country                          620027 non-null object
dtypes: datetime64[ns](1), float64(2), object(2)
memory usage: 28.4+ MB


In [18]:
# Rename columns as short column names
gw_2_df = gw_2_df.rename(columns={'dt':'date',
                 'AverageTemperature':'avg_temp_c',
                 'AverageTemperatureUncertainty':'temp_uncertainty'})

In [19]:
# Reset the index to break the connection to the old data frame
gw_2_df = gw_2_df.sort_values(["date"], ascending=[True]).\
                              reset_index(drop=True)

In [20]:
# Convert Celsius to Fahrenheit and reorder columns
gw_2_df["avg_temp_f"] = gw_2_df["avg_temp_c"].\
                        apply(lambda x: ((x * 1.8)+32))
gw_2_df = gw_2_df[["date","avg_temp_f","avg_temp_c",
                   "temp_uncertainty","State","Country"]]
gw_2_df.head(5)

,date,avg_temp_f,avg_temp_c,temp_uncertainty,State,Country
0,1743-11-01,30.5042,-0.831,3.427,Wisconsin,United States
1,1743-11-01,42.7154,5.953,2.220,Maryland,United States
2,1743-11-01,28.2056,-2.108,2.507,Novgorod,Russia
3,1743-11-01,21.7256,-5.708,4.134,Tatarstan,Russia
4,1743-11-01,20.8238,-6.209,3.532,Murmansk,Russia


In [21]:
# drop data for 1743 because there is only one month
gw_2_df = gw_2_df.loc[gw_2_df["date"] >= "1744-01-01"]

In [22]:
# Reset the index to break the connection to the old data frame
gw2_min_date = gw_2_df["date"].min()
gw2_max_date = gw_2_df["date"].max()
print("min date " + str(gw2_min_date) + " max date " + \
      str(gw2_max_date))

min date 1744-04-01 00:00:00 max date 2013-09-01 00:00:00


In [23]:
gw_2_df.info()
gw_2_df.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 619947 entries, 80 to 620026
Data columns (total 6 columns):
date                619947 non-null datetime64[ns]
avg_temp_f          619947 non-null float64
avg_temp_c          619947 non-null float64
temp_uncertainty    619947 non-null float64
State               619947 non-null object
Country             619947 non-null object
dtypes: datetime64[ns](1), float64(3), object(2)
memory usage: 33.1+ MB


,date,avg_temp_f,avg_temp_c,temp_uncertainty,State,Country
80,1744-04-01,51.6128,10.896,3.857,Iowa,United States
81,1744-04-01,73.3604,22.978,2.470,Florida,United States
82,1744-04-01,58.0388,14.466,3.482,Missouri,United States
83,1744-04-01,45.2804,7.378,2.350,Massachusetts,United States
84,1744-04-01,66.8120,19.340,2.724,Georgia (State),United States


In [45]:
# Export file as an XLSX or CSV, w/o index, w/ header
gw_2_df.to_csv("./output/clean_GlobalLandTemperaturesByState.csv", index=False, header=True)

In [25]:
# GlobalLandTemperatures_GlobalLandTemperaturesByMajorCity.csv
# 
gw_3 = "./data/source/GlobalLandTemperatures_GlobalLandTemperaturesByMajorCity.csv"
# Read our Data file with the pandas library
gw_3_df = pd.read_csv(gw_3,parse_dates=["dt"])
gw_3_df.info()
gw_3_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 239177 entries, 0 to 239176
Data columns (total 7 columns):
dt                               239177 non-null datetime64[ns]
AverageTemperature               228175 non-null float64
AverageTemperatureUncertainty    228175 non-null float64
City                             239177 non-null object
Country                          239177 non-null object
Latitude                         239177 non-null object
Longitude                        239177 non-null object
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 12.8+ MB


,dt,AverageTemperature,AverageTemperatureUncertainty,City,Country,Latitude,Longitude
0,1849-01-01,26.704,1.435,Abidjan,Côte D'Ivoire,5.63N,3.23W
1,1849-02-01,27.434,1.362,Abidjan,Côte D'Ivoire,5.63N,3.23W
2,1849-03-01,28.101,1.612,Abidjan,Côte D'Ivoire,5.63N,3.23W
3,1849-04-01,26.140,1.387,Abidjan,Côte D'Ivoire,5.63N,3.23W
4,1849-05-01,25.427,1.200,Abidjan,Côte D'Ivoire,5.63N,3.23W


In [26]:
# Drop rows with NaN
gw_3_df= gw_3_df.dropna(axis=0)
gw_3_df.describe()
gw_3_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 228175 entries, 0 to 239175
Data columns (total 7 columns):
dt                               228175 non-null datetime64[ns]
AverageTemperature               228175 non-null float64
AverageTemperatureUncertainty    228175 non-null float64
City                             228175 non-null object
Country                          228175 non-null object
Latitude                         228175 non-null object
Longitude                        228175 non-null object
dtypes: datetime64[ns](1), float64(2), object(4)
memory usage: 13.9+ MB


In [27]:
# Rename columns as short column names
gw_3_df = gw_3_df.rename(columns={'dt':'date',
                 'AverageTemperature':'avg_temp_c',
                 'AverageTemperatureUncertainty':'temp_uncertainty'})

In [28]:
# Convert Celsius to Fahrenheit and reorder columns
gw_3_df["avg_temp_f"] = gw_3_df["avg_temp_c"].\
                        apply(lambda x: ((x * 1.8)+32))
gw_3_df = gw_3_df[["date","avg_temp_f","avg_temp_c",
                   "temp_uncertainty","City","Country",\
                   "Latitude","Longitude"]]
gw_3_df.head(5)

,date,avg_temp_f,avg_temp_c,temp_uncertainty,City,Country,Latitude,Longitude
0,1849-01-01,80.0672,26.704,1.435,Abidjan,Côte D'Ivoire,5.63N,3.23W
1,1849-02-01,81.3812,27.434,1.362,Abidjan,Côte D'Ivoire,5.63N,3.23W
2,1849-03-01,82.5818,28.101,1.612,Abidjan,Côte D'Ivoire,5.63N,3.23W
3,1849-04-01,79.0520,26.140,1.387,Abidjan,Côte D'Ivoire,5.63N,3.23W
4,1849-05-01,77.7686,25.427,1.200,Abidjan,Côte D'Ivoire,5.63N,3.23W


In [29]:
# Reset the index to break the connection to the old data frame
gw_3_df = gw_3_df.sort_values(["date"],  ascending=[True]).\
                              reset_index(drop=True)

In [30]:
# drop data for 1743 because there is only one month
gw_3_df = gw_3_df.loc[gw_3_df["date"] >= "1744-01-01"]

In [31]:
# Get the min and max date range
gw3_min_date = gw_3_df["date"].min()
gw3_max_date = gw_3_df["date"].max()
print("min date " + str(gw3_min_date) + " max date " + \
      str(gw3_max_date))

min date 1744-04-01 00:00:00 max date 2013-09-01 00:00:00


In [32]:
gw_3_df.info()
gw_3_df.head(5)

<class 'pandas.core.frame.DataFrame'>
Int64Index: 228162 entries, 13 to 228174
Data columns (total 8 columns):
date                228162 non-null datetime64[ns]
avg_temp_f          228162 non-null float64
avg_temp_c          228162 non-null float64
temp_uncertainty    228162 non-null float64
City                228162 non-null object
Country             228162 non-null object
Latitude            228162 non-null object
Longitude           228162 non-null object
dtypes: datetime64[ns](1), float64(3), object(4)
memory usage: 15.7+ MB


,date,avg_temp_f,avg_temp_c,temp_uncertainty,City,Country,Latitude,Longitude
13,1744-04-01,53.8772,12.154,2.006,Rome,Italy,42.59N,13.09E
14,1744-04-01,52.1402,11.189,2.170,Madrid,Spain,40.99N,4.26W
15,1744-04-01,46.9328,8.296,2.501,London,United Kingdom,52.24N,0.00W
16,1744-04-01,42.6056,5.892,2.249,Toronto,Canada,44.20N,80.50W
17,1744-04-01,37.8842,3.269,3.220,Saint Petersburg,Russia,60.27N,29.19E


In [46]:
# For data analysis: Export file as an XLSX or CSV, w/o index, w/ header
gw_3_df.to_csv("./output/clean_GlobalLandTemperaturesByMajorCity.csv", index=False, header=True)

In [34]:
#GlobalLandTemperatures_GlobalTemperatures.csv
# 
gw_4 = "./data/source/GlobalLandTemperatures_GlobalTemperatures.csv"
# Read our Data file with the pandas library
gw_4_df = pd.read_csv(gw_4,parse_dates=["dt"])
gw_4_df.info()
gw_4_df.head()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3192 entries, 0 to 3191
Data columns (total 9 columns):
dt                                           3192 non-null datetime64[ns]
LandAverageTemperature                       3180 non-null float64
LandAverageTemperatureUncertainty            3180 non-null float64
LandMaxTemperature                           1992 non-null float64
LandMaxTemperatureUncertainty                1992 non-null float64
LandMinTemperature                           1992 non-null float64
LandMinTemperatureUncertainty                1992 non-null float64
LandAndOceanAverageTemperature               1992 non-null float64
LandAndOceanAverageTemperatureUncertainty    1992 non-null float64
dtypes: datetime64[ns](1), float64(8)
memory usage: 224.5 KB


,dt,LandAverageTemperature,LandAverageTemperatureUncertainty,LandMaxTemperature,LandMaxTemperatureUncertainty,LandMinTemperature,LandMinTemperatureUncertainty,LandAndOceanAverageTemperature,LandAndOceanAverageTemperatureUncertainty
0,1750-01-01,3.034,3.574,NaN,NaN,NaN,NaN,NaN,NaN
1,1750-02-01,3.083,3.702,NaN,NaN,NaN,NaN,NaN,NaN
2,1750-03-01,5.626,3.076,NaN,NaN,NaN,NaN,NaN,NaN
3,1750-04-01,8.490,2.451,NaN,NaN,NaN,NaN,NaN,NaN
4,1750-05-01,11.573,2.072,NaN,NaN,NaN,NaN,NaN,NaN


In [35]:
# Drop rows with NaN
gw_4_df= gw_4_df.dropna(axis=0)
gw_4_df.describe()
gw_4_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 1992 entries, 1200 to 3191
Data columns (total 9 columns):
dt                                           1992 non-null datetime64[ns]
LandAverageTemperature                       1992 non-null float64
LandAverageTemperatureUncertainty            1992 non-null float64
LandMaxTemperature                           1992 non-null float64
LandMaxTemperatureUncertainty                1992 non-null float64
LandMinTemperature                           1992 non-null float64
LandMinTemperatureUncertainty                1992 non-null float64
LandAndOceanAverageTemperature               1992 non-null float64
LandAndOceanAverageTemperatureUncertainty    1992 non-null float64
dtypes: datetime64[ns](1), float64(8)
memory usage: 155.6 KB


In [36]:
# Rename columns as short column names
gw_4_df = gw_4_df.rename(columns={'dt':'date',
     'LandAverageTemperature':'land_avg_temp_c',
     'LandAverageTemperatureUncertainty':'land_avg_temp_uncertain',
     'LandMaxTemperature': 'land_max_temp_c',
     'LandMaxTemperatureUncertainty': 'land_max_temp_uncertain',
     'LandMinTemperature': 'land_min_temp_c',
     'LandMinTemperatureUncertainty': 'land_min_temp_uncertain',
     'LandAndOceanAverageTemperature': 'land_ocean_avg_temp_c',
     'LandAndOceanAverageTemperatureUncertainty': 'land_ocean_avg_temp_uncertain'})

In [37]:
# Reset the index to break the connection to the old data frame
gw_4_df = gw_4_df.sort_values(["date"], ascending=[True]).\
                               reset_index(drop=True)
gw_4_df.head()

,date,land_avg_temp_c,land_avg_temp_uncertain,land_max_temp_c,land_max_temp_uncertain,land_min_temp_c,land_min_temp_uncertain,land_ocean_avg_temp_c,land_ocean_avg_temp_uncertain
0,1850-01-01,0.749,1.105,8.242,1.738,-3.206,2.822,12.833,0.367
1,1850-02-01,3.071,1.275,9.970,3.007,-2.291,1.623,13.588,0.414
2,1850-03-01,4.954,0.955,10.347,2.401,-1.905,1.410,14.043,0.341
3,1850-04-01,7.217,0.665,12.934,1.004,1.018,1.329,14.667,0.267
4,1850-05-01,10.004,0.617,15.655,2.406,3.811,1.347,15.507,0.249


In [38]:
# Convert Celsius to Fahrenheit and reorder columns
gw_4_df["land_avg_temp_f"] = gw_4_df["land_avg_temp_c"].\
                        apply(lambda x: ((x * 1.8)+32))
gw_4_df["land_max_temp_f"] = gw_4_df["land_max_temp_c"].\
                        apply(lambda x: ((x * 1.8)+32))
gw_4_df["land_min_temp_f"] = gw_4_df["land_min_temp_c"].\
                        apply(lambda x: ((x * 1.8)+32))
gw_4_df["land_ocean_avg_temp_f"] = gw_4_df["land_ocean_avg_temp_c"].\
                        apply(lambda x: ((x * 1.8)+32))

    
gw_4_df = gw_4_df[["date","land_avg_temp_f","land_avg_temp_c",
                   "land_avg_temp_uncertain",
                   "land_max_temp_f","land_max_temp_c",
                   "land_max_temp_uncertain",
                   "land_min_temp_f","land_min_temp_c",
                   "land_min_temp_uncertain",
                   "land_ocean_avg_temp_f","land_ocean_avg_temp_c",
                   "land_ocean_avg_temp_uncertain"]]
gw_4_df.head(5)

,date,land_avg_temp_f,land_avg_temp_c,land_avg_temp_uncertain,land_max_temp_f,land_max_temp_c,land_max_temp_uncertain,land_min_temp_f,land_min_temp_c,land_min_temp_uncertain,land_ocean_avg_temp_f,land_ocean_avg_temp_c,land_ocean_avg_temp_uncertain
0,1850-01-01,33.3482,0.749,1.105,46.8356,8.242,1.738,26.2292,-3.206,2.822,55.0994,12.833,0.367
1,1850-02-01,37.5278,3.071,1.275,49.9460,9.970,3.007,27.8762,-2.291,1.623,56.4584,13.588,0.414
2,1850-03-01,40.9172,4.954,0.955,50.6246,10.347,2.401,28.5710,-1.905,1.410,57.2774,14.043,0.341
3,1850-04-01,44.9906,7.217,0.665,55.2812,12.934,1.004,33.8324,1.018,1.329,58.4006,14.667,0.267
4,1850-05-01,50.0072,10.004,0.617,60.1790,15.655,2.406,38.8598,3.811,1.347,59.9126,15.507,0.249


In [39]:
# Reset the index to break the connection to the old data frame
gw4_min_date = gw_4_df["date"].min()
gw4_max_date = gw_4_df["date"].max()
print("min date " + str(gw4_min_date) + " max date " + \
      str(gw4_max_date))

min date 1850-01-01 00:00:00 max date 2015-12-01 00:00:00


In [40]:
gw_4_df.info()
gw_4_df.head(5)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1992 entries, 0 to 1991
Data columns (total 13 columns):
date                             1992 non-null datetime64[ns]
land_avg_temp_f                  1992 non-null float64
land_avg_temp_c                  1992 non-null float64
land_avg_temp_uncertain          1992 non-null float64
land_max_temp_f                  1992 non-null float64
land_max_temp_c                  1992 non-null float64
land_max_temp_uncertain          1992 non-null float64
land_min_temp_f                  1992 non-null float64
land_min_temp_c                  1992 non-null float64
land_min_temp_uncertain          1992 non-null float64
land_ocean_avg_temp_f            1992 non-null float64
land_ocean_avg_temp_c            1992 non-null float64
land_ocean_avg_temp_uncertain    1992 non-null float64
dtypes: datetime64[ns](1), float64(12)
memory usage: 202.4 KB


,date,land_avg_temp_f,land_avg_temp_c,land_avg_temp_uncertain,land_max_temp_f,land_max_temp_c,land_max_temp_uncertain,land_min_temp_f,land_min_temp_c,land_min_temp_uncertain,land_ocean_avg_temp_f,land_ocean_avg_temp_c,land_ocean_avg_temp_uncertain
0,1850-01-01,33.3482,0.749,1.105,46.8356,8.242,1.738,26.2292,-3.206,2.822,55.0994,12.833,0.367
1,1850-02-01,37.5278,3.071,1.275,49.9460,9.970,3.007,27.8762,-2.291,1.623,56.4584,13.588,0.414
2,1850-03-01,40.9172,4.954,0.955,50.6246,10.347,2.401,28.5710,-1.905,1.410,57.2774,14.043,0.341
3,1850-04-01,44.9906,7.217,0.665,55.2812,12.934,1.004,33.8324,1.018,1.329,58.4006,14.667,0.267
4,1850-05-01,50.0072,10.004,0.617,60.1790,15.655,2.406,38.8598,3.811,1.347,59.9126,15.507,0.249


In [42]:
# For data analysis: Export file as an XLSX or CSV, w/o index, w/ header
gw_4_df.to_csv("./output/clean_GlobalLandTemps.csv", index=False, header=True)